# Lecture 6 - 05.07.2021

# Exercise 1: The InSAR Coherence & Height Inversion

* TanDEM-X acquisition over Mondah forest (Gabon)
* Path: './data/03-insar/'

In [1]:
# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *

%matplotlib widget


/projects/src/ste_io.py:69: UserWarning: Failed to import STEtools.visualisation. write_pixmap will not work.
  warnings.warn("Failed to import STEtools.visualisation. write_pixmap will not work.")


In [2]:
def calculate_correlation(im1, im2, looksr, looksa) : 
    
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    return corr

**Input parameters**

In [3]:
# path 2 images
path = './data/03-insar/'

# Input TDX pixel spacing, in meters
spacrg = 1.36411
spacaz = 1.86817

# TDX vertical wavenumber, in rad / meters
kz = 0.06092195

# Output range resolution, in meters
resrg = 20.
resaz = 20.



**Step 1: Open images, and visualize**

In [4]:
# --- open images
im1 = rrat(path + 'master_HH_Mondah_S_2015_11_11_cut.rat')
im2 = rrat(path + 'slave_HH_Mondah_S_2015_11_11_cut.rat')

# --- visulize amplitude
amp1 = np.sqrt(np.abs(calculate_correlation(im1, im1, 7, 7)))

plt.figure( figsize = (10,5) )
plt.imshow(amp1, vmin = 0, vmax = 2*np.mean(amp1), aspect = 'auto', cmap = 'gray')
plt.title('Amplitude')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Amplitude')

**Step 2 : Compensate flat-earth**

In [5]:
# --- read fe from file
fe = rrat(path + 'flat_earth_Mondah_S_2015_11_11_cut.rat')

# --- number of looks
looksr = int(resrg / spacrg)
if looksr % 2 == 0 : looksr = looksr +1
looksa = int(resaz / spacaz)
if looksa % 2 == 0 : looksa = looksa +1
    
# inteferogram
interf = np.angle(calculate_correlation(im1, im2, looksa, looksr))

plt.figure( figsize = (10,5) )
plt.imshow(interf, vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet')
plt.title('Interferogram')
plt.colorbar()

# --- compensate flat earth
im2_fe = im2 * np.exp(1j * fe)

# inteferogram
interf = np.angle(calculate_correlation(im1, im2_fe, looksa, looksr))

plt.figure( figsize = (10,5) )
plt.imshow(interf, vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet')
plt.title('Interferogram after FE')
plt.colorbar()

    



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 3 : Calculate coherence**

In [6]:
# --- Calculate coherence
amp1 = np.abs(calculate_correlation(im1, im1, looksa, looksr))
amp2 = np.abs(calculate_correlation(im2_fe, im2_fe, looksa, looksr))
gamma = calculate_correlation(im1, im2_fe, looksa, looksr)
gamma = gamma / np.sqrt(amp1) / np.sqrt(amp2)

plt.figure( figsize = (10,5) )
plt.imshow(np.abs(gamma), vmin = 0, vmax = 1, aspect = 'auto', cmap = 'gray')
plt.title('Coherence')
plt.colorbar()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 4 : Invert for forest height (closed-form)**

In [7]:
# --- compensate system decorrelation
gamma = np.abs(gamma) / 0.97
gamma[gamma > 1] = 1

# --- height inversion, v1
hv_1 = np.sqrt( 24 / (kz**2) * (1 - gamma) ) 

# --- height inversion
hv_2 = 2*np.pi/kz * (1-2/np.pi*np.arcsin(gamma**.8))




plt.figure( figsize = (10,5) )
plt.imshow(hv_1, vmin = 0, vmax = 60, aspect = 'auto', cmap = 'jet')
plt.title('Inverted forest height v1')
plt.colorbar()

plt.figure( figsize = (10,5) )
plt.imshow(hv_2, vmin = 0, vmax = 60, aspect = 'auto', cmap = 'jet')
plt.title('Inverted forest height v2')
plt.colorbar()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 5 : Validation**

In [8]:
# --- open lidar top height

RH100 = rrat(path + 'RH100_Mondah_S_2015_11_11_cut.rat')

plt.figure( figsize = (10,5) )
plt.imshow(RH100, vmin = 0, vmax = 60, aspect = 'auto', cmap = 'jet')
plt.title('Lidr top height')
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# 2-D validation histogram

hist2d_hv2 = np.histogram2d(RH100[RH100 > -9999], hv_2[RH100 > -9999], bins = 120, \
                                  range = [[0, 60], [0, 60]])

plt.figure( figsize = (8,8))
plt.imshow(np.flipud(np.transpose(hist2d_hv2[0])), aspect = 1, \
          extent = [0, 60, 0, 60], cmap = 'jet')
plt.plot((0, 60), (0, 60), lw = 2, color = 'w', linestyle = '--')
plt.xlabel('Lidar height (m)')
plt.ylabel('TDX - v2 (m)')
plt.grid(True, linestyle = '--', color = 'w')
plt.title('Height estimates')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Height estimates')